

## What is it?

This is an implementation of [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life). 

This is a refactored version of [an earlier attempt](https://github.com/ariesunique/python-shorts/blob/master/Game%20of%20Life.ipynb).

The main difference between this version and the earlier one is that I removed several of the helper functions. Although readability was my primary goal, my colleagues correctly pointed out that too many helper functions can also impede readability as it obscures the actual functionality of the code. The biggest change is that I removed a lot of the ugly repetition in the get_neighbors function.

### Simplified rules

The above logic can be simplified as follows:

We are primarily concerned with cells that will change state: 
* a cell that is ALIVE can become DEAD  
* a cell that is DEAD can become ALIVE

What are the conditions that cause a cell to change state?
* A cell that is ALIVE becomes DEAD if it has less than 2 live neighbors or more than 3 live neighbors.
* A cell that is DEAD becomes ALIVE if it has exactly 3 LIVE neighbors


## Implementation

### Defining a grid

We will maintain state in an n x n matrix.

For an n x n array, our indices will go from 1 .. n, with (1,1) being the cell in the top-left corner and (n,n) being the cell in the bottom-right corner

```
    (1,1)   (1,2)  ...  (1,n)
    
    (2,1)   (2,2)  ...  (2,n)
    
      |       |            |
      
    (n,1)   (n,2)  ...  (n, n)
```

A particular cell in the matrix can have one of two states: ON (aka ALIVE) or OFF (aka DEAD). These can be represented as any constants of your choice.

### Helper functions

We'll create some helper methods to manage our game board.

**__init__**

```
For convenience, there will be several ways to initialize a board. 
* If you provide the SIZE, you will get an n x n array (where n=SIZE) with all the cells initialized to the DEAD state.
* You can provide INIT, which is an n x n array, representing the exact initial state of the game
* Either SIZE or INIT must be provided for initialization.
* User cnn also optionally specify a list of COORDS representing the cells that should be turned on (ie, set the state of the cells specified in COORDS to ALIVE). 
```

**get_neighbors**

```
This will return a list of neighbors for a given cell. If the LIVEONLY flag is set, this method will include only neighbors that have a current state of ALIVE. (LIVEONLY is False by default, meaning you get the full list of neighbors if that flag is not specified).
```

**walk**

```
This is a generator that provides a convenient way to iterate over all the cells in the board. An optional CELL paramter can be provided which represents the starting cell (default is (1,1), ie the first cell in the upper-left corner of the board). Beginning with a given cell, the code will move to the right as far as possible, then move to the first cell in the column below the current cell. So cells will be returned from right to left, top to bottom (the same order that we read English). This method continues until it reaches the last cell in the grid.
```

**The full code for the Grid class is shown below**

In [1]:
class Grid:

    # Constants representing possible neighbors (named for convenience)
    ABOVE, BELOW, RIGHT, LEFT, UPPER_LEFT, UPPER_RIGHT, LOWER_LEFT, LOWER_RIGHT = [i for i in range(8)]

    # map of potential neighbors and the offset to the current cell
    possible_neighbors = {
        ABOVE: (-1, 0),
        BELOW: (1, 0),
        RIGHT: (0, 1),
        LEFT: (0, -1),
        UPPER_LEFT: (-1, -1),
        UPPER_RIGHT: (-1, 1),
        LOWER_LEFT: (1, -1),
        LOWER_RIGHT: (1, 1)
    }
    

    """
    size: len of the NxN array
    init: initial array (required if size is not given)
    coords: list of tuples (x,y) coords that should be turned ON
    consts: a tuple of strings representing the char to use to represent ON and OFF when printed
    """    
    def __init__(self, size=0, init=None, coords=None, consts=None):
        # hook so that we can easily change the display constants
        self.ON, self.OFF = 'X', '_'
        
        if consts:
            self.ON, self.OFF = consts
        
        # set the size of our array
        if init:
            self.size = len(init)
        elif size > 0:
            self.size = size
        else:
            raise Exception("An error occurred initializing the grid")
        
        # initialize our array to all dead cells
        rows = []
        for i in range(self.size):
            row = []
            for j in range(self.size):
                row.append(self.OFF)
            rows.append(row)
        self.grid = rows 
        
        # if an initial array was provided, copy the values into the grid
        if init:
            n = len(init); m = len(init[0])
            for i in range(n):
                for j in range(m):
                    self.grid[i][j] = init[i][j]
                    
            
        # if coords were provided, set these cells to ON
        if coords:
            for cell in coords:
                self.set_value(cell, self.ON)                
                    
    
    def __contains__(self, cell):
        # Note - we will use indices starting at 1 (not 0)
        x, y = cell
        return x > 0 and x <= self.size and y > 0 and y <= self.size
    
    
    def __str__(self):
        res = []
        for row in self.grid:
            res.append(" ".join(row))
        return "\n".join(res)
 

    def __len__(self):
        return self.size
        
    
    def set_value(self, cell, val):
        """Set a given cell to a specified state - ON or OFF """
        # NOTE: whenever we check the cell value we must adjust for zero-based indexing (front-end indexes from 1)
        x, y = cell
        self.grid[x-1][y-1] = val

        
    def is_alive(self, cell):
        """Check if a given cell is ALIVE"""
        # NOTE: whenever we check the cell value we must adjust for zero-based indexing (front-end indexes from 1)
        x, y = cell
        return self.grid[x-1][y-1] == self.ON
    
    
    def is_dead(self, cell):
        """Check if a given cell iS DEAD"""
        return not self.is_alive(cell)
    
      
    def get_neighbors(self, cell, liveonly=False):
        """
        Return a list of tuples representing all possible neighboring cells
        if LIVEONLY is set to True, only include neighbors who are alive in this list
        """        
        neighbors = []
        for offset in self.possible_neighbors.values():
            neighbor = (cell[0] + offset[0], cell[1] + offset[1])
            if self.__contains__(neighbor) and (not liveonly or self.is_alive(neighbor) ) :
                neighbors.append(neighbor)
           
        return neighbors

    
    def walk(self, cell=(1,1)):
        """Given a starting cell, iterate over the remaining cells in the grid, moving right, and then down, row by row"""
        current_row = cell[0]
        current_cell = cell
        while current_row <= self.size:
            while self.__contains__(current_cell):
                yield current_cell
                x, y = current_cell
                offsetx, offsety = self.possible_neighbors[self.RIGHT] 
                current_cell = ( x+offsetx, y+offsety )
            # if we get here, we've gone as far RIGHT in this grid as possible; 
            # so move down to the next row, and go back to the first column
            current_row += 1
            current_cell = (current_row, 1)
           

### Game logic

**The general flow of the game is as follows:**
* Initialize a game board (aka grid)
* Loop forever (or for a specified number of iterations)
  * get the new state of the grid
  * display the grid

**How do we get the new state of the grid**
* Iterate over every cell in the grid
  * Check the number of live neighbors the current cell has
  * Update the state of that cell according to the rules
  
*Note - we cannot actually change the state of a given cell until we have examined every cell. If we update the state of a cell as we iterate, then the new state will affect our processing of the next cell, which would not be accurate. Thus, we will maintain the new cell states in a new grid, and replace the original grid with the new grid once we have processed all the cells.*


In [2]:
def update_cell_state(newgrid, oldgrid, cell):
    """
    A cell is ALIVE in the next iteration if 
        it is ALIVE in the current iteration and has 2 or 3 live neighbors OR
        it is DEAD in the current iteration and has exactly 3 LIVE neighbors
    """
    
    # get number of live neighbors for this cell
    num_live_neighbors = len(oldgrid.get_neighbors(cell, liveonly=True))
    
    # if the cell meets the criteria for being alive, set the value in the new grid;
    # else the cell is dead 
    if (oldgrid.is_alive(cell) and (num_live_neighbors == 2 or num_live_neighbors == 3)) or \
       (oldgrid.is_dead(cell) and num_live_neighbors==3):
        
        newgrid.set_value(cell, newgrid.ON)  
    else:
        newgrid.set_value(cell, newgrid.OFF)  
        

def update_grid_state(grid):
    """Return a new grid representing the new state"""
    
    # initialize a new grid that is the same size as the initial array
    newgrid = Grid(size=len(grid), consts=(grid.ON, grid.OFF))
    
    # iterate over every cell in our input grid, and store the new state of the cell in the new grid
    for cell in grid.walk():
        update_cell_state(newgrid, grid, cell)
        
    # return the newgrid    
    return newgrid


def play_game(grid,iters=5):
    print("Initial grid")    
    print(grid)

    for i in range(iters):
        grid = update_grid_state(grid)
        print(f"\nGrid after iteration {i+1}")
        print(grid)

[top](#Table-of-Contents)

### Simple examples

**EXAMPLE 1**

Here's a grid that will change state only once, and then it gets stuck in a stable state.

In [3]:
grid = Grid(size=3, coords=[(1,1),(2,2),(3,3),(1,3),(3,1)])
play_game(grid, iters=3)

Initial grid
X _ X
_ X _
X _ X

Grid after iteration 1
_ X _
X _ X
_ X _

Grid after iteration 2
_ X _
X _ X
_ X _

Grid after iteration 3
_ X _
X _ X
_ X _


### Better examples

We will test our game using some of the patterns described on the [Wikipedia page](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life#Examples_of_patterns). 

For the examples below, I will use the clear_output function so that each iteration of the grid gets printed in the same place as the previous iteration. This will enable us to more easily observe how the state changes.

To make this work we need to modify our play_game function a bit.

In [4]:
from time import sleep
from IPython.display import clear_output, display

def play_game_inplace(grid,iters=5):
    print("\nInitial grid")    
    print(grid)
    sleep(1)

    for i in range(iters):
        grid = update_grid_state(grid)
        clear_output(wait=True)
        print(f"\nGrid after iteration {i+1}")
        print(grid)
        sleep(1)

In [5]:
# left, bottom, top, right
box1 = [(5,3),(6,3),(7,3),(8,5),(8,6),(8,7),(3,5),(3,6),(3,7),(5,8),(6,8),(7,8)]
box2 = [(5,10),(6,10),(7,10),(8,11),(8,12),(8,13),(3,11),(3,12),(3,13),(5,15),(6,15),(7,15)]
box3 = [(11,3),(12,3),(13,3),(15,5),(15,6),(15,7),(10,5),(10,6),(10,7),(11,8),(12,8),(13,8)]
box4 = [(11,10),(12,10),(13,10),(15,11),(15,12),(15,13),(10,11),(10,12),(10,13),(11,15),(12,15),(13,15)]

coords = []
coords.extend(box1)
coords.extend(box2)
coords.extend(box3)
coords.extend(box4)

grid = Grid(size=17, coords=coords, consts=['O','_'])
play_game_inplace(grid, iters=22)


Grid after iteration 22
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ O O O _ _ O O _ O O _ _ O O O _
_ _ _ O _ O _ O _ O _ O _ O _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ O _ O _ O _ O _ O _ O _ _ _
_ O O O _ _ O O _ O O _ _ O O O _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ O O _ _ _ O O _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ O _ _ _ _ _ O _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _


## Can we discover something new?

Let's try with completely random input and see if anything interesting happens.

In [6]:
from random import randint

grid_size = randint(3, 20)
num_inputs = randint(0, grid_size**2)
coords = []
for i in range(num_inputs):
    x = randint(0, grid_size-1)
    y = randint(0, grid_size-1)
    coords.append((x,y))
#print(f"Generating board of size {grid_size}, with {num_inputs} inputs.\nHere are the initial coordinates that are ALIVE: {coords}\n")

grid = Grid(size=grid_size, coords=coords, consts=['O','_'])
play_game_inplace(grid, iters=50)

print(f"\n\nGenerated board of size {grid_size}, with {num_inputs} inputs.\nHere are the initial coordinates that were ALIVE: {coords}\n")



Grid after iteration 50
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ O _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ O _ O _ _ _ _ _ _ _ _ _ _
_ O O _ O O _ O _ _ _ _ _ _ _ O O _
O _ _ O _ _ O _ _ _ _ _ _ _ O _ _ O
O _ O _ _ _ _ _ _ _ _ _ _ _ O _ _ O
_ O _ _ _ _ _ _ _ _ _ _ _ _ _ O O _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ O O O _ _ _ _ _ _ _ _
_ _ O O _ _ O _ _ _ O _ _ _ _ _ _ _
_ O _ _ O _ O _ _ O _ _ _ _ _ _ _ _
_ _ O O _ _ _ O O _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ O O _ _
_ _ _ _ _ _ _ _ _ _ _ _ _ _ O O _ _


Generated board of size 18, with 203 inputs.
Here are the initial coordinates that were ALIVE: [(11, 17), (12, 8), (10, 1), (3, 3), (7, 3), (0, 15), (7, 13), (8, 14), (13, 17), (12, 13), (5, 16), (13, 13), (7, 2), (0, 3), (17, 3), (4, 13), (4, 17), (5, 12), (7, 10), (0, 13), (1, 0), (4, 10), (2, 15), (7, 7), (17, 10), (15, 